In [86]:
%pip install librosa --user

In [87]:
import xgboost
from sklearn.model_selection import train_test_split, cross_val_predict
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import librosa
import numpy as np


import warnings
warnings.filterwarnings('ignore')
# ## create the classifier object leaving all params to default values except random_state
sk_dt = DecisionTreeClassifier(random_state=1)


In [88]:
df = pd.read_csv(r'data\Data\features_3_sec.csv')

# df = df.drop(['filename'], axis=1)

# df = df[['chroma_stft_mean','chroma_stft_var','rms_mean','rms_var','spectral_centroid_mean','spectral_centroid_var','spectral_bandwidth_mean','spectral_bandwidth_var','label']]

df = df[['chroma_stft_mean','chroma_stft_var','rms_mean','rms_var','spectral_centroid_mean','spectral_centroid_var','spectral_bandwidth_mean','spectral_bandwidth_var','rolloff_mean','rolloff_var','zero_crossing_rate_mean','zero_crossing_rate_var','harmony_mean','harmony_var','tempo','label']]
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()


df['label'] =  label_encoder.fit_transform(df['label'])

print(label_encoder.classes_)

y = df[['label']]
X = df[df.columns.difference(['label'])]


## split both X and y using a ratio of 70% training - 30% testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)
print(len(X_train), len(X_test), len(y_train), len(y_test))



['blues' 'classical' 'country' 'disco' 'hiphop' 'jazz' 'metal' 'pop'
 'reggae' 'rock']
6993 2997 6993 2997


In [89]:


xgb = xgboost.XGBClassifier(n_estimators=1000,enable_catergorical=True,learning_rate=0.05)
xgb.fit(X_train, y_train)

## make predictions on the test portion (predict the labels of the rows from the test portion of X)
predictions = xgb.predict(X_test)

target_name = ['blues', 'classical', 'country', 'disco', 'hiphop' ,'jazz' ,'metal', 'pop','reggae' ,'rock']


print(classification_report(y_test, predictions, target_names=target_name))
## can also output the confusion matrix
# cm = confusion_matrix(y_test, predictions)
# print(cm)

print("Accuracy: " ,metrics.accuracy_score(y_test, predictions))

cols_when_model_builds = xgb.feature_names_in_


              precision    recall  f1-score   support

       blues       0.77      0.83      0.80       302
   classical       0.91      0.92      0.92       296
     country       0.68      0.76      0.72       312
       disco       0.78      0.78      0.78       283
      hiphop       0.85      0.79      0.82       320
        jazz       0.80      0.81      0.80       289
       metal       0.87      0.89      0.88       303
         pop       0.84      0.77      0.80       298
      reggae       0.78      0.73      0.75       310
        rock       0.65      0.61      0.63       284

    accuracy                           0.79      2997
   macro avg       0.79      0.79      0.79      2997
weighted avg       0.79      0.79      0.79      2997

Accuracy:  0.7911244577911245


# Find Similar

In [90]:
df.head()

,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,rolloff_mean,rolloff_var,zero_crossing_rate_mean,zero_crossing_rate_var,harmony_mean,harmony_var,tempo,label
0,0.335406,0.091048,0.130405,0.003521,1773.065032,167541.630869,1972.744388,117335.771563,3714.560359,1.080790e+06,0.081851,0.000558,-0.000078,0.008354,129.199219,0
1,0.343065,0.086147,0.112699,0.001450,1816.693777,90525.690866,2010.051501,65671.875673,3869.682242,6.722448e+05,0.087173,0.001030,-0.000099,0.004950,123.046875,0
2,0.346815,0.092243,0.132003,0.004620,1788.539719,111407.437613,2084.565132,75124.921716,3997.639160,7.907127e+05,0.071383,0.000425,-0.000066,0.012476,123.046875,0
3,0.363639,0.086856,0.132565,0.002448,1655.289045,111952.284517,1960.039988,82913.639269,3568.300218,9.216524e+05,0.069426,0.000304,-0.000014,0.008318,123.046875,0
4,0.335579,0.088129,0.143289,0.001701,1630.656199,79667.267654,1948.503884,60204.020268,3469.992864,6.102111e+05,0.070095,0.000289,0.000041,0.009634,123.046875,0


In [105]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder

def find_sim(data):
    df_sim = pd.read_csv(r'data\Data\features_3_sec.csv')

    label_encoder = LabelEncoder()
    df_sim['label'] =  label_encoder.fit_transform(df_sim['label'])


    numeric_features = df_sim[['chroma_stft_mean','chroma_stft_var','rms_mean','rms_var','spectral_centroid_mean','spectral_centroid_var','spectral_bandwidth_mean','spectral_bandwidth_var','rolloff_mean','rolloff_var','zero_crossing_rate_mean','zero_crossing_rate_var','harmony_mean','harmony_var','tempo','label']]

    similarities = cosine_similarity(data, numeric_features)


    similar_rows = np.where(similarities >= 0.9)[1]

    sorted_rows = np.argsort(similarities[0, similar_rows])[::-1]

    for row_index in sorted_rows[:3]:
        similarity_index = similarities[0, row_index]

        print("Filename:", df_sim.iloc[row_index]['filename'], "  | Sim: ", similarity_index)
            
        print(df_sim.iloc[row_index])
        print('================================================================')


In [106]:
# import pandas as pd
# df = pd.read_csv('data\Data\musicFeatuers.csv')

# from sklearn.preprocessing import LabelEncoder
# label_encoder = LabelEncoder()

# # df['label'] =  label_encoder.fit_transform(df['label'])

# df = df.drop(['filename'], axis=1)



# X = df[['chroma_stft_mean','chroma_stft_var','rms_mean','rms_var','spectral_centroid_mean','spectral_centroid_var','spectral_bandwidth_mean','spectral_bandwidth_var']]

# y = df['label']

# y = df[['label']]
# X = df[df.columns.difference(['label'])]

# from sklearn.model_selection import train_test_split

# # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=1)

# X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.3, random_state=1)
# ## train a tree model
# sk_dt.fit(X_train, y_train)

# from sklearn.linear_model import LinearRegression
# # model = LinearRegression()
# # model.fit(X_train, y_train)


# from sklearn.metrics import classification_report

# predictions = sk_dt.predict(X_test)


# print(classification_report(y_test, predictions))

# y_pred = sk_dt.predict(X_test)



# from sklearn.metrics import mean_squared_error
# mse = mean_squared_error(Y_test,y_pred)
# print(mse)


# Feature Extractor

** 'blues':0, 'classical':1, 'country':2, 'disco':3, 'hiphop':4 ,'jazz':5 ,'metal':6, 'pop':7,'reggae':8 ,'rock':9 **

In [107]:
def extract_features(file):
    y, sr = librosa.load(file)

    chroma_sft_mean =  np.mean(librosa.feature.chroma_stft(y=y, sr=sr))
    chroma_sft_var =  librosa.feature.chroma_stft(y=y, sr=sr).var()

    rms_mean = librosa.feature.rms(y=y).mean()
    rms_var = librosa.feature.rms(y=y).var()

    spectral_centroid_mean = librosa.feature.spectral_centroid(y=y, sr=sr).mean()
    spectral_centroid_var = librosa.feature.spectral_centroid(y=y, sr=sr).var()

    spectral_bandwith_mean = librosa.feature.spectral_bandwidth(y=y, sr=sr).mean()
    spectral_bandwith_var = librosa.feature.spectral_bandwidth(y=y, sr=sr).var()

    rolloff_mean = librosa.feature.spectral_rolloff(y=y, sr=sr).mean()
    rolloff_var = librosa.feature.spectral_rolloff(y=y, sr=sr).var()

    zero_crossing_rate_mean = librosa.feature.zero_crossing_rate(y=y).mean()
    zero_crossing_rate_var = librosa.feature.zero_crossing_rate(y=y).var()


    harmony_mean = librosa.effects.harmonic(y).mean()
    harmony_var = librosa.effects.harmonic(y).var()

    tempo = librosa.feature.tempo(y=y, sr=sr)[0]

    # print('---------------------------------------------------')
    # print("chroma_sft_mean: ",chroma_sft_mean)
    # print("chroma_sft_var: ",chroma_sft_var)
    # print('---------------------------------------------------')
    # print("rms_mean: ",rms_mean)
    # print("rms_var: ",rms_var)
    # print('---------------------------------------------------')
    # print("spectral_centroid_mean: ",spectral_centroid_mean)
    # print("spectral_centroid_var: ",spectral_centroid_var)
    # print('---------------------------------------------------')
    # print("spectral_bandwith_mean: ",spectral_bandwith_mean)
    # print("spectral_bandwith_var: ",spectral_bandwith_var)
    # print('---------------------------------------------------')
    # print("rolloff_mean: ",rolloff_mean)
    # print("rolloff_var: ",rolloff_var)
    # print('---------------------------------------------------')
    # print("zero_crossing_rate_mean: ",zero_crossing_rate_mean)
    # print("zero_crossing_rate_var: ",zero_crossing_rate_var)
    # print('---------------------------------------------------')
    # print("harmony_mean: ",harmony_mean)
    # print("harmony_var: ",harmony_var)
    # print('---------------------------------------------------')
    # print("tempo: ",tempo)

    features = pd.DataFrame({'chroma_stft_mean':[chroma_sft_mean],'chroma_stft_var':[chroma_sft_var],'rms_mean':[rms_mean],'rms_var':[rms_var],'spectral_centroid_mean':[spectral_centroid_mean],
                             'spectral_centroid_var':[spectral_centroid_var],'spectral_bandwidth_mean':[spectral_bandwith_mean],'spectral_bandwidth_var':[spectral_bandwith_var],
                             'rolloff_mean':[rolloff_mean],'rolloff_var':[rolloff_var],'zero_crossing_rate_mean':[zero_crossing_rate_mean],'zero_crossing_rate_var':[zero_crossing_rate_var],
                             'harmony_mean':[harmony_mean],'harmony_var':[harmony_var],'tempo':[tempo],})
    

    features = features.reindex(columns=cols_when_model_builds)

    # features = features.reshape(-1,1)
    return features

features1=  extract_features(r"data\Data\genres_original\pop\pop.00023.wav")
# features1 = extract_features(r"music/juice.mp3")
# features2 =  extract_features(r"music\newmoonlight.mp3")


genre1 = xgb.predict(features1)

# # genre2 = xgb.predict(features2)

print('---------------------------------------------------')

print("1: ",genre1[0])
# print("2: ",genre2[0])

features1['label'] = genre1[0]

find_sim(features1)

---------------------------------------------------
1:  7
Filename: blues.00000.0.wav   | Sim:  0.14692428099704796
filename                   blues.00000.0.wav
length                                 66149
chroma_stft_mean                    0.335406
chroma_stft_var                     0.091048
rms_mean                            0.130405
rms_var                             0.003521
spectral_centroid_mean           1773.065032
spectral_centroid_var          167541.630869
spectral_bandwidth_mean          1972.744388
spectral_bandwidth_var         117335.771563
rolloff_mean                     3714.560359
rolloff_var                   1080789.885581
zero_crossing_rate_mean             0.081851
zero_crossing_rate_var              0.000558
harmony_mean                       -0.000078
harmony_var                         0.008354
perceptr_mean                      -0.000068
perceptr_var                        0.005535
tempo                             129.199219
mfcc1_mean                   